# Functor

Functor is a function that takes a module and output a module - function from module to module.

##### 🟢 &nbsp; &nbsp; Syntax

In [36]:
module type T = sig
    val x : int
end

module F(M: T) = struct
    let x = M.x + 1
end

module M = struct let x = 7 end

module Incr = F(M) ;;
Incr.x ;;

module type T = sig val x : int end


module F : functor (M : T) -> sig val x : int end


module M : sig val x : int end


module Incr : sig val x : int end


- : int = 8


In [37]:
module G(M : sig val x : float end) = struct
    let x = M.x -. 1.
end

module Decr = G(struct let x = 7. end) ;;
Decr.x ;;

module G : functor (M : sig val x : float end) -> sig val x : float end


module Decr : sig val x : float end


- : float = 6.


In [38]:
module type X = sig val x : int end ;;
module type Y = sig val y :int end ;;

module Make (X' : X) (Y' : Y ) = struct
    let x = X'.x + 1
    let y = Y'.y + 1
end ;;

module R = Make(struct let x = 1 end) (struct let y = 2 end) ;;

R.x ;;
R.y ;;


module type X = sig val x : int end


module type Y = sig val y : int end


module Make : functor (X' : X) (Y' : Y) -> sig val x : int val y : int end


module R : sig val x : int val y : int end


- : int = 2


- : int = 3


In [39]:
module type Num = sig   
    val x : int
    val y : int
end ;;

module type X = sig val x : int end ;;
module type Y = sig val y : int end ;;

module type Foo = sig
    val xplusy : int
end

(* contravariant ? *)
module Make : Num -> Num -> Foo = functor (X' : X) -> functor (Y' : Y) -> struct
    let xplusy = X'.x + Y'.y 
end

module R  = Make(struct let x = 4 let y = 5 end) (struct let x = 4 let y = 5 end) ;;
R.xplusy ;;

module type Num = sig val x : int val y : int end


module type X = sig val x : int end


module type Y = sig val y : int end


module type Foo = sig val xplusy : int end


module Make : Num -> Num -> Foo


module R : Foo


- : int = 9


In [40]:
module type S = sig
  val foo : int -> int
end

module O (X : S) = struct
  let cow x = X.foo x
  let sheep x = 1 + cow x
end [@@inline always]

module F (X : S) (Y : S) = struct
  let cow x = Y.foo (X.foo x)
  let sheep x = 1 + cow x
end [@@inline always]

module type S1 = sig
  val bar : int -> int
  val foo : int -> int
end

module type T = sig
  val sheep : int -> int
end

module F1 (X : S) (Y : S) : T = struct
  let cow x = Y.foo (X.foo x)
  let sheep x = 1 + cow x
end [@@inline always]

module F2 : S1 -> S1 -> T = functor (X : S) -> functor (Y : S) -> struct
  let cow x = Y.foo (X.foo x)
  let sheep x = 1 + cow x
end [@@inline always]

(* functor F is contravariant in S and covariant in T ? *)
module M : sig
  module F (X : S1) (Y : S1) : T
end = struct
  module F (X : S) (Y : S) = struct
    let cow x = Y.foo (X.foo x)
    let sheep x = 1 + cow x
  end [@@inline always]
end

module R = M.F (struct let foo x = x let bar x = x end) (struct let foo x = x let bar x = x end ) ;;
R.sheep 2 ;;
(* R.cow 2 ;; won't work *)

module type S = sig val foo : int -> int end


module O :
  functor (X : S) -> sig val cow : int -> int val sheep : int -> int end


module F :
  functor (X : S) (Y : S) ->
    sig val cow : int -> int val sheep : int -> int end


module type S1 = sig val bar : int -> int val foo : int -> int end


module type T = sig val sheep : int -> int end


module F1 : functor (X : S) (Y : S) -> T


module F2 : S1 -> S1 -> T


module M : sig module F : functor (X : S1) (Y : S1) -> T end


module R : T


- : int = 3


More

In [41]:
(* Like in JavaScript, 0 is falsy, greater/less than 0 are truthy. *)
module type JSBOOL = sig
    type t
    val equal : t -> t -> bool
    val zero : t 
end

module Make (M: JSBOOL) = struct
    type t = M.t
    let torf v = if (M.equal v M.zero) then false else true  
end

(* above is same as 
    module Make = functor (M: JSBOOL) -> struct
        type t = M.t
        let torf v = if (M.equal v M.zero) then false else true  
    end
*)

module BoolOfInt = Make(Int) ;;
let open BoolOfInt in
torf 0;;

module BoolOfFloat = Make(Float) ;;
let open BoolOfFloat in
torf 1.0;; 

module BoolOfIntPair = 
    Make(struct
      type t = int * int
      let zero = (0, 0)
      let equal p1 p2 = p1 = p2
  end) ;;
    
let open BoolOfIntPair in
torf (0, 1);;

module BoolOfString = 
    Make(struct
        type t = string
        let zero = ""
        let equal = String.equal
  end) ;;

open BoolOfString;;
torf "hello";;
torf "";;

module type JSBOOL = sig type t val equal : t -> t -> bool val zero : t end


module Make :
  functor (M : JSBOOL) -> sig type t = M.t val torf : M.t -> bool end


module BoolOfInt : sig type t = Int.t val torf : Int.t -> bool end


- : bool = false


module BoolOfFloat : sig type t = Float.t val torf : Float.t -> bool end


...

module BoolOfIntPair :
  sig type t = int * int val torf : int * int -> bool end


- : bool = true


module BoolOfString : sig type t = string val torf : string -> bool end


- : bool = true


- : bool = false


<br/>

##### 🟢 &nbsp; &nbsp; Sample code

In [42]:
module type Addable = sig
    type t
    val add : t -> t -> t    
end

module type Subtractable = sig
    type t
    val sub : t -> t -> t    
end

module type Comparable = sig
    type t
    val compare : t -> t -> int
end

module type Show = sig
    type t
    val to_string : t -> string
end

module type ADD_SUB_CMP_SH = sig
    type t
    include Addable with type t := t
    include Subtractable with type t := t
    include Comparable with type t := t
    include Show with type t := t
end

module type POINT = sig
    type item
    type t
    include ADD_SUB_CMP_SH with type t := t
    val make : item -> item -> t
    val getX : t -> item
    val getY : t -> item 
    val ( + ) : t -> t -> t
    val ( - ) : t -> t -> t
    val ( > ) : t -> t -> bool
    val ( < ) : t -> t -> bool
    val ( >= ) : t -> t -> bool
    val ( <= ) : t -> t -> bool    
end

module Make (M: ADD_SUB_CMP_SH) : POINT with type item := M.t = struct
    type item = M.t
    type t = item * item
    let make x y = (x, y)
    let getX (x, _) = x
    let getY (_, y) = y
    let add (x1, y1) (x2, y2) = (M.add x1 x2, M.add y1 y2)
    let sub (x1, y1) (x2, y2) = (M.sub x1 x2, M.sub y1 y2)
    let compare (x1, y1) (x2, y2) = 
        match M.compare x1 x2 with
        | 1 -> 1
        | -1 -> -1
        | _ -> 
            match M.compare y1 y2 with
            | 0 -> 0
            | c -> c
    let ( + ) = add
    let ( - ) = sub
    let ( > ) p1 p2 = if compare p1 p2 = 1 then true else false
    let ( < ) p1 p2 = if compare p1 p2 = -1 then true else false
    let ( >= ) p1 p2 = match compare p1 p2 with 1 | 0 -> true | _ -> false
    let ( <= ) p1 p2 = match compare p1 p2 with -1 | 0 -> true | _ -> false
    let to_string (x, y) = Printf.sprintf "(x = %s, y = %s)" (M.to_string x) (M.to_string y)        
end

module PointInt = Make(Int) ;;
open PointInt;;
let p1 = make 1 1;;
let p2 = make 2 2;;
(p1 + p2) |> to_string;;
p1 > p2 ;;
p1 <= p2 ;;

module PointInt = Make(Float) ;;
let open PointInt in
let p1 = make 10. 10. 
and p2 = make 2. 2. in
(p1 - p2) |> to_string;;

module type Addable = sig type t val add : t -> t -> t end


module type Subtractable = sig type t val sub : t -> t -> t end


module type Comparable = sig type t val compare : t -> t -> int end


module type Show = sig type t val to_string : t -> string end


module type ADD_SUB_CMP_SH =
  sig
    type t
    val add : t -> t -> t
    val sub : t -> t -> t
    val compare : t -> t -> int
    val to_string : t -> string
  end


...

- : string = "(x = 3, y = 3)"


- : bool = false


- : bool = true


module PointInt :
  sig
    type t = Make(Float).t
    val add : t -> t -> t
    val sub : t -> t -> t
    val compare : t -> t -> int
    val to_string : t -> string
    val make : Float.t -> Float.t -> t
    val getX : t -> Float.t
    val getY : t -> Float.t
    val ( + ) : t -> t -> t
    val ( - ) : t -> t -> t
    val ( > ) : t -> t -> bool
    val ( < ) : t -> t -> bool
    val ( >= ) : t -> t -> bool
    val ( <= ) : t -> t -> bool
  end


- : string = "(x = 8., y = 8.)"


Sample Code

In [15]:
module type Comparable = sig
    type t
    val compare : t -> t -> int
end ;;

type 'a comparable = (module Comparable with type t = 'a) ;;
type 'a repr = { cmp : 'a -> 'a -> int; data : 'a list } ;;
let make (type a) (c : a comparable) =
    let module C = (val c) in
    { cmp = C.compare; data = [] } ;;
    
make (module struct  type t = int let compare x y = Stdlib.compare x y end) ;;

module type Comparable = sig type t val compare : t -> t -> int end


type 'a comparable = (module Comparable with type t = 'a)


type 'a repr = { cmp : 'a -> 'a -> int; data : 'a list; }


val make : 'a comparable -> 'a repr = <fun>


- : int repr = {cmp = <fun>; data = []}
